# Notebook de prise en main du package reclin2

Ce notebook vise à présenter le fonctionnement du package R reclin2 et les possibilités qu'il offre. Le package reclin2 est un outil  permettant de mettre en oeuvre des appariements probabilistes via la méthode de Fellegi et Sunter. Il propose également diverses fonctions auxiliaires pour faciliter la mise en œuvre de toutes les étapes d'un appariement.

Les données utilisées sont des données synthétiques issues du [générateur de données synthétiques du package FEBRL](https://users.cecs.anu.edu.au/~Peter.Christen/Febrl/febrl-0.3/febrldoc-0.3/node70.html). Deux fichiers sont disponibles. Le premier fichier contient 5000 individus fictifs dont les traits d'identité (nom, prénom, date de naissance, etc.) proviennent de tables de fréquence tirés d'annuaires téléphoniques australiens. Le second fichier contient les mêmes 5000 individus, mais avec des erreurs sur les traits d'identité (substitution de caractères, suppression d'une valeur, remplacement par des variantes communes d'orthographe, etc.).

La documentation du package reclin2 est disponible [ici](https://cran.r-project.org/web/packages/reclin2/reclin2.pdf). Des tutoriels officiels sont également accessibles depuis [la page github du package](https://github.com/djvanderlaan/reclin2).

Les différents termes techniques et notions évoquées dans ce notebook sont définis dans le document de travail associé.

## Import des librairies et chargement des données

In [1]:
### Installation du package
install.packages('reclin2')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [2]:
### Import des packages nécessaires
library(reclin2)
library(plyr)

Loading required package: data.table


Attaching package: ‘reclin2’


The following object is masked from ‘package:base’:

    identical




Si vous ne disposez pas déjà des données, il faut décommenter et exécuter les lignes de code ci-dessous pour les télécharger.

In [3]:
### Téléchargement des données
#url <- "https://raw.githubusercontent.com/InseeFrLab/appariement/main/data/data_febrl_5000_A.csv"
#destfile <- "../data/data_febrl_5000_A.csv"
#download.file(url, destfile)
#url <- "https://raw.githubusercontent.com/InseeFrLab/appariement/main/data/data_febrl_5000_B.csv"
#destfile <- "../data/data_febrl_5000_B.csv"
#download.file(url, destfile)

In [4]:
### Chargement des données
dfA <- read.csv('data_febrl_5000_A.csv')
dfB <- read.csv('data_febrl_5000_B.csv')

In [5]:
n <- nrow(dfA)

On peut visualiser les 5 premières lignes des deux fichiers pour observer leur structure.

In [6]:
head(dfA)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
1,rec-1070-org,michaela,neumann,8,stanley street,miami,winston hills,4223,nsw,19151111,5304218
2,rec-1016-org,courtney,painter,12,pinkerton circuit,bega flats,richlands,4560,vic,19161214,4066625
3,rec-4405-org,charles,green,38,salkauskas crescent,kela,dapto,4566,nsw,19480930,4365168
4,rec-1288-org,vanessa,parr,905,macquoid place,broadbridge manor,south grafton,2135,sa,19951119,9239102
5,rec-3585-org,mikayla,malloney,37,randwick road,avalind,hoppers crossing,4552,vic,19860208,7207688
6,rec-298-org,blake,howie,1,cutlack street,belmont park belted galloway stud,budgewoi,6017,vic,19250301,5180548


In [7]:
head(dfB)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>
1,rec-561-dup-0,elton,,3,light setreet,pinehill,windermere,3212,vic,19651013,1551941
2,rec-2642-dup-0,mitchell,maxon,47,edkins street,lochaoair,north ryde,3355,nsw,19390212,8859999
3,rec-608-dup-0,,white,72,lambrigg street,kelgoola,broadbeach waters,3159,vic,19620216,9731855
4,rec-3239-dup-0,elk i,menzies,1,lyster place,,northwood,2585,vic,19980624,4970481
5,rec-2886-dup-0,,garanggar,NA,may maxwell crescent,springettst arcade,forest hill,2342,vic,19921016,1366884
6,rec-4285-dup-0,sophie,manson,14,elizabeth xrescent,manorhouse,gorokan,3465,vic,19510201,4818868


## Appariement étape par étape

### Préparation des données

En général, la première étape d'un appariement consiste en une succession de traitements pour nettoyer les données. Ici, les données synthétiques utilisées sont déjà dans un format satisfaisant (uniquement des caractères minuscules, pas d'accents, etc.). On va donc simplement calculer de nouvelles variables à partir de celles déjà existantes.

La variable "année de naissance", calculée à partir de la date de naissance, sera utile pour l'étape de réduction du nombre de paires.

In [8]:
### Ajout d'une variable année de naissance
dfA$birthyear <- as.numeric(substr(dfA$date_of_birth, start = 1, stop = 4))
dfB$birthyear <- as.numeric(substr(dfB$date_of_birth, start = 1, stop = 4))

Puisque ces données synthétiques ont été générées spécialement pour tester des appariements, on connaît et on peut retrouver toutes les paires d'individus identiques. Pour évaluer la qualité de l'appariement à la fin du processus, il sera utile d'avoir un identifiant commun pour les deux fichiers.

En effet, les identifiants dans les deux fichiers ont des formes légèrement différentes. Pour le fichier A, l'identifiant prend la forme "rec-\*\*\*\*-org" ; tandis que pour le fichier B, il se présente comme suit : "rec-\*\*\*\*-dup-0".
On crée un identifiant commun aux deux fichiers en ne conservant que "rec-\*\*\*\*"

In [9]:
### Création d'un identifiant commun 
dfA$rec_id_stripped <- substr(dfA$rec_id, 1, nchar(dfA$rec_id)-4)
dfB$rec_id_stripped <- substr(dfB$rec_id, 1, nchar(dfB$rec_id)-6)

Il est utile de compter les valeurs manquantes dans chaque colonne afin de déterminer quels champs sont affectés et si le nombre de valeurs manquantes est élevé ou négligeable. Ici, les champs `street_number`, `postcode` et `date_of_birth` possèdent des valeurs manquantes.

In [10]:
### Comptage des valeurs manquantes par colonne
sapply(dfA, function(x) sum(is.na(x)))
sapply(dfB, function(x) sum(is.na(x)))

rec_id      given_name         surname   street_number       address_1 
              0               0               0             158               0 
      address_2          suburb        postcode           state   date_of_birth 
              0               0               0               0              94 
     soc_sec_id       birthyear rec_id_stripped 
              0              94               0

rec_id      given_name         surname   street_number       address_1 
              0               0               0             287               0 
      address_2          suburb        postcode           state   date_of_birth 
              0               0               0               0             199 
     soc_sec_id       birthyear rec_id_stripped 
              0             199               0

### Réduction du nombre de paires

Il s'agit maintenant de choisir les paires qui vont être comparées. Une première possibilité est de considérer l'ensemble des paires du produit cartésien des deux fichiers.

In [11]:
pairs <- pair(dfA, dfB)

In [12]:
### Conservation de toutes les paires
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 25 000 000 pairs

            .x   .y
       1:    1    1
       2:    1    2
       3:    1    3
       4:    1    4
       5:    1    5
      ---          
24999996: 5000 4996
24999997: 5000 4997
24999998: 5000 4998
24999999: 5000 4999
25000000: 5000 5000


En utilisant cette stratégie sur les deux fichiers d'exemples qui contiennent 5000 lignes, on obtient 25 millions de paires. Dans un cas réel, les fichiers sont souvent plus gros, il devient alors impossible de conserver toutes les paires. Il faut donc trouver un moyen de filtrer. La manière la plus classique, et celle proposée par le package reclin2, est le blocage.

#### Blocage

Le blocage consiste à conserver uniquement les paires qui correspondent sur une variable ou une sélection de variables, appelées "clés de blocage". Une bonne clé de blocage doit être assez discriminante pour réduire largement la dimension, et de très bonne qualité pour éviter de supprimer des paires d'individus identiques.

Dans le package reclin2, le blocage s'effectue via la fonction _pair_blocking_. L'objet retourné contient l'ensemble des paires retenues, ainsi que quelques informations supplémentaires comme le nombre total de paires conservées.

In [13]:
### Blocage simple sur le code postal
pairs <- pair_blocking(dfA, dfB, "postcode")
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 28 609 pairs
  Blocking on: 'postcode'

         .x   .y
    1: 2158 2239
    2: 2158 3020
    3: 3389 2239
    4: 3389 3020
    5:  700 1091
   ---          
28605: 3480 4384
28606: 3763  500
28607: 3763 2797
28608: 3763 4384
28609: 4568 1692


Il est possible de combiner plusieurs clés de blocage avec une condition "ET" en utilisant une liste de variables. Ici, les paires conservées sont celles qui correspondent à la fois sur les variables `postcode` et `birthyear`.

In [14]:
### Blocage sur le code postal ET l'année de naissance
pairs <- pair_blocking(dfA, dfB, c("postcode", "birthyear"))
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 4 139 pairs
  Blocking on: 'postcode', 'birthyear'

        .x   .y
   1: 2158 3020
   2: 3389 2239
   3: 4429 1447
   4: 3934 2235
   5: 1929 1091
  ---          
4135: 1533 4021
4136: 3155 2797
4137: 3763  500
4138: 3480 4384
4139: 4568 1692


Pour combiner les clés de blocage avec une condition "OU", il faut utiliser la fonction _merge_pairs_. Ici, les paires conservées sont celles qui correspondent soit sur la variable `postcode`, soit sur la variable `birthyear` (ou sur les deux).

In [15]:
### Blocage sur le code postal OU la date de naissance
pairs <- merge_pairs(
  pair_blocking(dfA, dfB, "postcode"),
  pair_blocking(dfA, dfB, "birthyear")
)
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 284 524 pairs

          .x   .y
     1:    1  268
     2:    1  279
     3:    1  280
     4:    1  442
     5:    1  574
    ---          
284520: 5000 4463
284521: 5000 4526
284522: 5000 4624
284523: 5000 4734
284524: 5000 4825


Pour la suite de ce tutoriel, nous allons utiliser un blocage simple sur le code postal.

In [16]:
### Blocage simple sur le code postal
pairs <- pair_blocking(dfA, dfB, "postcode")
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 28 609 pairs
  Blocking on: 'postcode'

         .x   .y
    1: 2158 2239
    2: 2158 3020
    3: 3389 2239
    4: 3389 3020
    5:  700 1091
   ---          
28605: 3480 4384
28606: 3763  500
28607: 3763 2797
28608: 3763 4384
28609: 4568 1692


### Comparaison des champs

L'étape suivante consiste à comparer les paires conservées champ par champ. La fonction _compare_pairs_ permet d'effectuer cette opération. Le package propose les méthodes de comparaison suivantes : 
- Exacte, via la fonction _identical_
- Jaro-Winkler, via la fonction _jaro_winkler_
- _Longest common substring_ (proche de la distance de Levenshtein), via la fonction _lcs_
- Jaccard, via la fonction _jaccard_

Hormis pour la comparaison exacte, ces méthodes de comparaison sont issues des fonctions correspondantes du package stringdist, à une différence près : elles sont transformées en des mesures de similarité plutôt que des distances.

Les mesures de similarité renvoient une valeur comprise entre 0 et 1, la valeur 1 indiquant une correspondance parfaite. La méthode de classification probabiliste mise en oeuvre dans ce package nécessite cependant des mesures de comparaison binaires. Il faut donc préciser un seuil, qui sera utilisé pour transformer les variables de comparaison continues, en des variables binaires 0-1. Par exemple, la fonction _jaro_winkler(0.92)_ permet d'effectuer une comparaison via la similarité de Jaro-Winkler et de binariser le résultat en utilisant la valeur seuil de 0.92.

Par défaut, ce sont des comparaisons exactes qui sont effectuées. La méthode de comparaison par défaut peut-être modifiée par l'intermédiaire du paramètre `default_comparator`. Il est également possible de spécifier une méthode de comparaison différente pour un ou plusieurs champs donnés, via le paramètre `comparators`.

Dans cet exemple, les champs `date_of_birth`, `suburb` et `state` sont comparés de façon exacte, avec un résultat binaire. Les champs `given_name` et `surname` sont comparés via la similarité de Jaro-Winkler, et le résultat est transformé en une variable binaire en fixant un seuil à 0.92. Enfin, c'est la similarité calculée à partir de la méthode de la _longest common substring_ qui est utilisée pour le champ `address_1`, avec un seuil à 0.85 pour binariser.

On n'effectue aucune comparaison sur la variable `postcode` ici, puisqu'elle est la seule clé de blocage : ainsi, toutes les paires retenues présentent forcément le même code postal, cette variable ne porte plus d'information.

In [17]:
### Comparaison des champs
compare_pairs(pairs, on = c("given_name", "surname", "date_of_birth", "suburb", "state", "address_1"), 
              comparators = list(given_name = jaro_winkler(), surname = jaro_winkler(0.92), address_1 = lcs(0.85)), 
              default_comparator = identical(), inplace = TRUE)

print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 28 609 pairs
  Blocking on: 'postcode'

         .x   .y given_name   surname date_of_birth suburb state address_1
    1: 2158 2239  0.5396825 0.5777778         FALSE  FALSE FALSE 0.2962963
    2: 2158 3020  1.0000000 0.8888889          TRUE   TRUE  TRUE 1.0000000
    3: 3389 2239  0.7527778 1.0000000          TRUE   TRUE  TRUE 1.0000000
    4: 3389 3020  0.7130952 0.5777778         FALSE  FALSE FALSE 0.2962963
    5:  700 1091  0.6666667 0.4365079         FALSE  FALSE FALSE 0.5294118
   ---                                                                    
28605: 3480 4384  0.4285714 0.4365079          TRUE   TRUE  TRUE 1.0000000
28606: 3763  500  1.0000000 0.4559885          TRUE   TRUE  TRUE 0.3846154
28607: 3763 2797  0.7037037 0.4642857         FALSE  FALSE  TRUE 0.6153846
28608: 3763 4384  0.5886243 0.5238095         FALSE  FALSE FALSE 0.4000000
28609: 4568 1692  1.0000000 1.0000000       

Lorsqu'une valeur est manquante, le résultat de la comparaison est aussi une valeur manquante. Cela posera un problème au moment de la classification des paires si ces valeurs sont laissées telles quelles. Ainsi, lorsqu'une valeur est manquante, on fixe la similarité à 0.

In [18]:
### Correction pour les valeurs manquantes
pairs$date_of_birth[is.na(pairs$date_of_birth)] <- FALSE

Voici le résultat de la comparaison des champs sur quelques individus. Les comparaisons n'ont été effectuées  et ne sont donc disponibles que sur les paires d'individus conservées à l'issue de l'étape d'indexation (c'est-à-dire ceux ayant le même code postal dans cet exemple).

In [19]:
### Définition d'une fonction pour afficher les mesures de similarité pour un individu donné

print_field_comparisons <- function(record_number) {
  tryCatch({
      # The pairs object uses row numbers as index: find row numbers based on record id
      x_rownumber <- which(attributes(pairs)$x$rec_id == paste0("rec-", record_number, "-org"))
      y_rownumber <- which(attributes(pairs)$y$rec_id == paste0("rec-", record_number, "-dup-0"))
      
      # Create dataframe with the given records from file A and B, 
      #and the correspondingcomparison measures
      df_comp <- t(do.call(plyr::rbind.fill, 
          list(dfA[dfA$rec_id==paste0("rec-", record_number, "-org"),], 
               dfB[dfB$rec_id==paste0("rec-", record_number, "-dup-0"),], 
              pairs[(pairs$.x == x_rownumber) & (pairs$.y == y_rownumber), ])))
      
      # Format the dataframe: add column names, remove unnecessary rows
      colnames(df_comp) <- df_comp["rec_id",]
      colnames(df_comp)[3] <- "Similarité"
      
      rows_to_keep <- c("given_name", "surname", "date_of_birth", "address_1", "suburb", "state", "postcode", "street_number", 
                  "address_2", "soc_sec_id", "birthyear")
      df_comp <- df_comp[rows_to_keep, ]
      
      # Add column with comparison mode
      comparison_modes <- data.frame(
    "Modes_de_comparaison" = 
    c("Jaro-Winkler", "Jaro-Winkler", "Exact", "Longest common substring", "Exact", "Exact",
      "Non comparé (clé de blocage)", "Non comparé", "Non comparé", "Non comparé", "Non comparé"),  
    row.names = rows_to_keep)

    cbind(df_comp, comparison_modes)
      
  }, error = function(e) {
    message("Les mesures de comparaison ne sont pas disponibles pour cet individu.")
    return(NULL)
  })
}

In [20]:
print_field_comparisons(1070)

,rec-1070-org,rec-1070-dup-0,Similarité,Modes_de_comparaison
,<chr>,<chr>,<chr>,<chr>
given_name,michaela,michafla,0.925925925925926,Jaro-Winkler
surname,neumann,jakimow,0.472222222222222,Jaro-Winkler
date_of_birth,19151111,19151111,1,Exact
address_1,stanley street,stanleykstreet,0.933333333333333,Longest common substring
suburb,winston hills,winstonbhills,FALSE,Exact
state,nsw,,FALSE,Exact
postcode,4223,4223,NA,Non comparé (clé de blocage)
street_number,8,8,NA,Non comparé
address_2,miami,miami,NA,Non comparé


In [21]:
print_field_comparisons(1124)

,rec-1124-org,rec-1124-dup-0,Similarité,Modes_de_comparaison
,<chr>,<chr>,<chr>,<chr>
given_name,lily,lily,1,Jaro-Winkler
surname,clarke,clarke,1,Jaro-Winkler
date_of_birth,19690612,19690612,1,Exact
address_1,emerton street,emerton street,1,Longest common substring
suburb,woodville south,woodville south,TRUE,Exact
state,nsw,nsw,TRUE,Exact
postcode,4223,4223,NA,Non comparé (clé de blocage)
street_number,25,25,NA,Non comparé
address_2,eight mile,eight yile,NA,Non comparé


In [22]:
print_field_comparisons(1016)

,rec-1016-org,rec-1016-dup-0,Similarité,Modes_de_comparaison
,<chr>,<chr>,<chr>,<chr>
given_name,courtney,courtney,1,Jaro-Winkler
surname,painter,painter,1,Jaro-Winkler
date_of_birth,19161214,19161214,1,Exact
address_1,pinkerton circuit,pinkerton circuit,1,Longest common substring
suburb,richlands,richlands,TRUE,Exact
state,vic,vci,FALSE,Exact
postcode,4560,4560,NA,Non comparé (clé de blocage)
street_number,12,12,NA,Non comparé
address_2,bega flats,,NA,Non comparé


### Comparaison des paires : classification

L'approche principale proposée par le package reclin2 pour la classification des paires est l'approche probabiliste, conforme au cadre de Fellegi et Sunter. La fonction _problink_em_ permet de procéder à l'estimation des paramètres du modèle, via l'algorithme itératif Espérance-Maximisation (EM).

In [23]:
### Estimation des paramètres du modèle
m <- problink_em(~given_name + surname + date_of_birth + address_1 + suburb + state, data = pairs)
print(m)

M- and u-probabilities estimated by the EM-algorithm:
      Variable M-probability U-probability
    given_name     0.7393867   0.004365923
       surname     0.8153348   0.005935374
 date_of_birth     0.8921841   0.000100000
     address_1     0.9050878   0.001885350
        suburb     0.7486620   0.001249271
         state     0.9501785   0.222184811

Matching probability: 0.1471533.


La fonction _predict_ permet ensuite d'utiliser le modèle qui vient d'être calibré afin d'attribuer une probabilité à chaque paire.

In [24]:
### Calcul d'une probabilité pour chaque paire
pairs <- predict(m, pairs = pairs, type = "mpost", add = TRUE)
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 28 609 pairs
  Blocking on: 'postcode'

         .x   .y given_name   surname date_of_birth suburb state address_1
    1: 2158 2239  0.5396825 0.5777778         FALSE  FALSE FALSE 0.2962963
    2: 2158 3020  1.0000000 0.8888889          TRUE   TRUE  TRUE 1.0000000
    3: 3389 2239  0.7527778 1.0000000          TRUE   TRUE  TRUE 1.0000000
    4: 3389 3020  0.7130952 0.5777778         FALSE  FALSE FALSE 0.2962963
    5:  700 1091  0.6666667 0.4365079         FALSE  FALSE FALSE 0.5294118
   ---                                                                    
28605: 3480 4384  0.4285714 0.4365079          TRUE   TRUE  TRUE 1.0000000
28606: 3763  500  1.0000000 0.4559885          TRUE   TRUE  TRUE 0.3846154
28607: 3763 2797  0.7037037 0.4642857         FALSE  FALSE  TRUE 0.6153846
28608: 3763 4384  0.5886243 0.5238095         FALSE  FALSE FALSE 0.4000000
28609: 4568 1692  1.0000000 1.0000000       

À partir de ces probabilités, il est alors possible de définir un seuil au-delà duquel on décide de lier la paire. Dans l'exemple ci-dessous, on lie toutes les paires dont la probabilité estimée est supérieure à 0.5, et on stocke cette décision sous la forme d'une variable booléenne appelée `above_threshold`.

In [25]:
### Attribution d'un statut à chaque paire (lié ou non-lié)
select_threshold(pairs, variable = "above_threshold", score = "mpost", threshold = 0.5, inplace = TRUE)
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 28 609 pairs
  Blocking on: 'postcode'

         .x   .y given_name   surname date_of_birth suburb state address_1
    1: 2158 2239  0.5396825 0.5777778         FALSE  FALSE FALSE 0.2962963
    2: 2158 3020  1.0000000 0.8888889          TRUE   TRUE  TRUE 1.0000000
    3: 3389 2239  0.7527778 1.0000000          TRUE   TRUE  TRUE 1.0000000
    4: 3389 3020  0.7130952 0.5777778         FALSE  FALSE FALSE 0.2962963
    5:  700 1091  0.6666667 0.4365079         FALSE  FALSE FALSE 0.5294118
   ---                                                                    
28605: 3480 4384  0.4285714 0.4365079          TRUE   TRUE  TRUE 1.0000000
28606: 3763  500  1.0000000 0.4559885          TRUE   TRUE  TRUE 0.3846154
28607: 3763 2797  0.7037037 0.4642857         FALSE  FALSE  TRUE 0.6153846
28608: 3763 4384  0.5886243 0.5238095         FALSE  FALSE FALSE 0.4000000
28609: 4568 1692  1.0000000 1.0000000       

### Résolution des conflits

L'approche d'appariement probabiliste, comme les autres méthodes usuelles de classification, traite les paires de façon indépendante. Cependant, lorsqu'on apparie des données individuelles et que les fichiers à apparier ne contiennent pas de doublons, on a besoin d'imposer une contrainte supplémentaire : un individu du premier fichier doit être apparié avec au plus un individu du second fichier. Il s'agit alors de filtrer les paires identifiées dans l'étape précédente afin de respecter cette contrainte.

Le package reclin2 propose deux approches pour résoudre les éventuels conflits : 
- la fonction `select_greedy` repose sur une approche dite "gloutonne" : les paires sont évaluées de façon séquentielle, en commençant par celles ayant la probabilité (estimée par le modèle) la plus élevée ;
- la fonction `select_n_to_m` cherche une solution optimale en maximisant la somme des probabilités associées à l'ensemble des paires retenues.

L'approche gloutonne est nettement plus rapide, et les solutions obtenues avec les deux méthodes sont souvent très proches.

In [26]:
### Résolution des conflits par l'approche gloutonne
select_greedy(pairs, variable = "selected_greedy", score = "mpost", threshold = 0.5, inplace = TRUE)
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 28 609 pairs
  Blocking on: 'postcode'

         .x   .y given_name   surname date_of_birth suburb state address_1
    1: 2158 2239  0.5396825 0.5777778         FALSE  FALSE FALSE 0.2962963
    2: 2158 3020  1.0000000 0.8888889          TRUE   TRUE  TRUE 1.0000000
    3: 3389 2239  0.7527778 1.0000000          TRUE   TRUE  TRUE 1.0000000
    4: 3389 3020  0.7130952 0.5777778         FALSE  FALSE FALSE 0.2962963
    5:  700 1091  0.6666667 0.4365079         FALSE  FALSE FALSE 0.5294118
   ---                                                                    
28605: 3480 4384  0.4285714 0.4365079          TRUE   TRUE  TRUE 1.0000000
28606: 3763  500  1.0000000 0.4559885          TRUE   TRUE  TRUE 0.3846154
28607: 3763 2797  0.7037037 0.4642857         FALSE  FALSE  TRUE 0.6153846
28608: 3763 4384  0.5886243 0.5238095         FALSE  FALSE FALSE 0.4000000
28609: 4568 1692  1.0000000 1.0000000       

In [27]:
### Résolution des conflits par l'approche optimale
select_n_to_m(pairs, variable = "selected_optimal", score = "mpost", threshold = 0.5, inplace = TRUE)
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 28 609 pairs
  Blocking on: 'postcode'

         .x   .y given_name   surname date_of_birth suburb state address_1
    1: 2158 2239  0.5396825 0.5777778         FALSE  FALSE FALSE 0.2962963
    2: 2158 3020  1.0000000 0.8888889          TRUE   TRUE  TRUE 1.0000000
    3: 3389 2239  0.7527778 1.0000000          TRUE   TRUE  TRUE 1.0000000
    4: 3389 3020  0.7130952 0.5777778         FALSE  FALSE FALSE 0.2962963
    5:  700 1091  0.6666667 0.4365079         FALSE  FALSE FALSE 0.5294118
   ---                                                                    
28605: 3480 4384  0.4285714 0.4365079          TRUE   TRUE  TRUE 1.0000000
28606: 3763  500  1.0000000 0.4559885          TRUE   TRUE  TRUE 0.3846154
28607: 3763 2797  0.7037037 0.4642857         FALSE  FALSE  TRUE 0.6153846
28608: 3763 4384  0.5886243 0.5238095         FALSE  FALSE FALSE 0.4000000
28609: 4568 1692  1.0000000 1.0000000       

### Évaluation de la qualité

Évaluer la qualité d'un appariement est une étape primordiale mais souvent difficile à réaliser car on ne connaît pas le vrai statut de chaque paire. Il faut donc utiliser un échantillon représentatif annoté.

Ici, la tâche est plus simple puisque nous travaillons avec des données générées. On peut donc calculer les résultats sur l'ensemble des individus. Le package n'a pas de fonctionnalités particulières liées à l'évaluation de la qualité, mais on peut calculer les mesures classiques en procédant à quelques opérations sur le dataframe `pairs`. La fonction _compute_performance_metrics_FEBRL_ proposée ci-dessous permet de calculer les mesures de performance pour les données utilisées dans ce notebook.

In [28]:
### Ajout du vrai statut de chaque paire
pairs <- compare_vars(pairs, "truth", on_x = "rec_id_stripped", on_y = "rec_id_stripped")
print(pairs)

  First data set:  5 000 records
  Second data set: 5 000 records
  Total number of pairs: 28 609 pairs
  Blocking on: 'postcode'

         .x   .y given_name   surname date_of_birth suburb state address_1
    1: 2158 2239  0.5396825 0.5777778         FALSE  FALSE FALSE 0.2962963
    2: 2158 3020  1.0000000 0.8888889          TRUE   TRUE  TRUE 1.0000000
    3: 3389 2239  0.7527778 1.0000000          TRUE   TRUE  TRUE 1.0000000
    4: 3389 3020  0.7130952 0.5777778         FALSE  FALSE FALSE 0.2962963
    5:  700 1091  0.6666667 0.4365079         FALSE  FALSE FALSE 0.5294118
   ---                                                                    
28605: 3480 4384  0.4285714 0.4365079          TRUE   TRUE  TRUE 1.0000000
28606: 3763  500  1.0000000 0.4559885          TRUE   TRUE  TRUE 0.3846154
28607: 3763 2797  0.7037037 0.4642857         FALSE  FALSE  TRUE 0.6153846
28608: 3763 4384  0.5886243 0.5238095         FALSE  FALSE FALSE 0.4000000
28609: 4568 1692  1.0000000 1.0000000       

In [29]:
### Définition d'une fonction pour calculer les mesures de qualité de l'appariement
compute_performance_metrics_FEBRL <- function(pairs, dataset_size, result_column = "selected_optimal") {
    TP <- sum(subset(pairs, select = result_column) & pairs$truth)
    FP <- sum(subset(pairs, select = result_column) & !pairs$truth)
    FN <- dataset_size - TP
    TN <- dataset_size * dataset_size - TP - FN - FP
    
    precision <- TP / (TP + FP)
    recall <- TP / (TP + FN)
    Fscore <- 2 * precision * recall / (precision + recall)
    return(list(TP = TP, TN = TN, FP = FP, FN = FN, precision = precision, recall = recall, Fscore = Fscore))
    }

In [30]:
### Résultats sans résolution des conflits
compute_performance_metrics_FEBRL(pairs, n, "above_threshold")

$TP
[1] 4213

$TN
[1] 24994940

$FP
[1] 60

$FN
[1] 787

$precision
[1] 0.9859583

$recall
[1] 0.8426

$Fscore
[1] 0.9086595

In [31]:
### Résultats avec résolution des conflits par l'approche gloutonne
compute_performance_metrics_FEBRL(pairs, n, "selected_greedy")

$TP
[1] 4213

$TN
[1] 24994999

$FP
[1] 1

$FN
[1] 787

$precision
[1] 0.9997627

$recall
[1] 0.8426

$Fscore
[1] 0.914478

In [32]:
### Résultats avec résolution des conflits par l'approche optimale
compute_performance_metrics_FEBRL(pairs, n, "selected_optimal")

$TP
[1] 4213

$TN
[1] 24994999

$FP
[1] 1

$FN
[1] 787

$precision
[1] 0.9997627

$recall
[1] 0.8426

$Fscore
[1] 0.914478

On peut effectuer deux constats sur les résultats. 
- D'abord, il n'y a aucun faux positif ou presque, probablement dû au fait que les erreurs générées sur le fichier B sont assez légères et ne perturbent donc pas trop l'algorithme d'appariement.
- Du côté des faux négatifs, c'est moins satisfaisant : on rate beaucoup de paires d'individus identiques. La raison de ce défaut est sans doute le blocage un peu trop strict qui a été choisi, puisqu'on n'utilise que le code postal comme clé de blocage. Ainsi, tous les individus qui présentent une erreur sur le code postal dans le fichier B n'ont aucune chance d'être bien appariés, quelle que soit la qualité des données sur les autres champs identifiants. On pourrait donc améliorer l'appariement en utilisant plusieurs clés de bocage, afin de ne pas mettre trop de poids sur une seule variable.

Quoi qu'il en soit, ces résultats sont propres aux deux fichiers appariés. Un autre jeu de fichiers, avec des données de qualité différente et des types d'erreurs différents, donnerait un tout autre résultat.

## Exemple de code complet pour un appariement

In [33]:
### Import des packages nécessaires
library(reclin2)
library(plyr)

### Chargement des données
dfA <- read.csv('data_febrl_5000_A.csv')
dfB <- read.csv('data_febrl_5000_B.csv')

### Blocage simple sur le code postal
pairs <- pair_blocking(dfA, dfB, "postcode")

### Comparaison des champs et correction pour les valeurs manquantes
compare_pairs(pairs, on = c("given_name", "surname", "date_of_birth", "suburb", "state", "address_1"), 
              comparators = list(given_name = jaro_winkler(), surname = jaro_winkler(0.92), address_1 = lcs(0.85)), 
              default_comparator = identical(), inplace = TRUE)
pairs$date_of_birth[is.na(pairs$date_of_birth)] <- FALSE

### Estimation des paramètres du modèle puis calcul d'une probabilité pour chaque paire
m <- problink_em(~given_name + surname + date_of_birth + address_1 + suburb + state, data = pairs)
pairs <- predict(m, pairs = pairs, type = "mpost", add = TRUE)

### Résolution des conflits par l'approche gloutonne
select_greedy(pairs, variable = "selected_greedy", score = "mpost", threshold = 0.5, inplace = TRUE)

## Exercice : Appariement de deux fichiers de 10 000 lignes

À vous de jouer ! Ci-dessous sont chargés deux fichiers de 10 000 lignes. La mission (ou du moins son intitulé) est simple : apparier ces deux fichiers et obtenir le meilleur F1-score possible (libre à vous de choisir une autre mesure à optimiser).

Ces deux fichiers ont été générés avec le même outil que les deux fichiers de 5000 lignes utilisés jusqu'ici dans le tutoriel du package, mais avec des règles différentes :
- les deux fichiers ne se recouvrent qu'à 90%, c'est-dire que 10% des individus du fichier A ne sont pas dans le fichier B, et 10% des individus du fichier B ne sont pas dans le fichier A ;
- la distribution des erreurs est différente.

Comme précédemment, il n'y a pas de doublon dans les fichiers, ce qui signifie qu'un individu peut être lié au maximum à un seul individu de l'autre fichier.

In [34]:
### Téléchargement des données
#url <- "https://raw.githubusercontent.com/InseeFrLab/appariement/main/data/synthetic_data_10000_A.csv"
#destfile <- "synthetic_data_10000_A.csv"
#download.file(url, destfile)
#url <- "https://raw.githubusercontent.com/InseeFrLab/appariement/main/data/synthetic_data_10000_B.csv"
#destfile <- "synthetic_data_10000_B.csv"
#download.file(url, destfile)

In [35]:
### Chargement des données
dfA <- read.csv('synthetic_data_10000_A.csv')
dfB <- read.csv('synthetic_data_10000_B.csv')

In [36]:
head(dfA, 3)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,date_of_birth,sorting_key
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<int>,<dbl>,<int>
1,rec-6058-org,robert,clarke,4,blythe close,brentwood vlge,devonport,3178,19510506,6058
2,rec-4926-org,melanie,garnett,9,byrne street,,mirrabooka,2870,NA,4926
3,rec-4724-org,david,white,17,halford crescent,,,7030,19300220,4724


In [37]:
head(dfB, 3)

,rec_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,date_of_birth,sorting_key
,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<int>
1,rec-2896-dup-0,zachazuah,miell,73,neeld place,,anglsea,3163,19850524,2896
2,rec-5417-dup-0,jagco,galarneau,55,dobell circuit,,sunshine,4261,19880618,5417
3,rec-3033-dup-0,fioan,zilm,8,sternberg cbescent,beth-el,redbank plains,2478,19479316,3033


In [38]:
### Création d'un identifiant commun 
dfA$rec_id_stripped <- substr(dfA$rec_id, 1, nchar(dfA$rec_id)-4)
dfB$rec_id_stripped <- substr(dfB$rec_id, 1, nchar(dfB$rec_id)-6)

### Conversion de la colonne postcode en type numérique pour le fichier B, 
### afin que le type soit le même que dans le fichier A
dfB$postcode <- as.numeric(dfB$postcode)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


La fonction ci-dessous permet de calculer les mesures de qualité de l'appariement pour ces deux fichiers.

In [39]:
### Définition d'une fonction pour calculer les mesures de qualité de l'appariement
### Il faut au préalable ajouter la colonne truth à l'objet pairs avec la ligne de code suivante :
#pairs <- compare_vars(pairs, "truth", on_x = "rec_id_stripped", on_y = "rec_id_stripped")

compute_performance_metrics_90_percent_overlap <- function(pairs, dataset_size, result_column = "selected_optimal") {
    TP <- sum(subset(pairs, select = result_column) & pairs$truth)
    FP <- sum(subset(pairs, select = result_column) & !pairs$truth)
    FN <- dataset_size - TP
    TN <- dataset_size * dataset_size - TP - FN - FP
    
    precision <- TP / (TP + FP)
    recall <- TP / (TP + FN)
    Fscore <- 2 * precision * recall / (precision + recall)
    return(list(TP = TP, TN = TN, FP = FP, FN = FN, precision = precision, recall = recall, Fscore = Fscore))
    }